我们需要了解一下ACG的数值在各个动作中的特点，因此我们演示了以下动作，并记录了ACG数据。每个动作进行了10s。在动作间有10s的静息期。

In [167]:
import datetime
import os.path

import mne.io
import pandas as pd

import dataset as ds

In [168]:
def acc_to_locomotion2(acc):
    loco = np.linalg.norm(acc, axis=0).reshape(1, -1)
    return loco

In [169]:
from convert_x8_data import acc_to_locomotion

path = os.path.join(ds.path['free_eeg'], 'test', 'acg')
raw = mne.io.read_raw_edf(os.path.join(path, 'action_acg_2.edf'), preload=True)

loco = acc_to_locomotion(raw.get_data()[-3:, :]) / 1e6
loco2 = acc_to_locomotion2(raw.get_data()[-3:, :]) / 1e6
loco2 = acc_to_locomotion2(raw.get_data()[-3:, :]) / 1e6
raw._data[1, :] = loco
# raw._data[0, :] = loco2
raw._data[2:, :] /= 1e6

raw.rename_channels({'EEG1': 'ACG'})
# raw.rename_channels({'EEG0': 'ACG2'})

Measurement date,"August 03, 2024 18:02:15 GMT"
Experimenter,Unknown
Participant,X
Digitized points,Not available
Good channels,5 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,515.00 Hz
Highpass,0.00 Hz
Lowpass,257.50 Hz


In [170]:
df_action = pd.read_excel(os.path.join(path, 'action_2.xlsx'))
df_action

,序号,动作,Action,开始时间,动作组
0,1,静止,Rest,18:03:20,NaN
1,2,蹲起,Squat and rise,18:04:00,10个
2,3,跳跃,Jump,18:04:40,10个
3,4,匀缓速直行,Go straight ahead at a steady speed,18:05:20,10*2
4,5,加速直行,Accelerating straight,18:06:20,3*7
5,6,绕圈,Circling,18:06:40,2*1 + 2*1
6,7,点头,Nod,18:07:40,5*4
7,8,摇头,Shake the head,18:08:20,5*3
8,9,忽然摆头,Suddenly turn the head,18:09:00,4*2+1
9,10,开合嘴巴,Open and close the mouth,18:09:40,NaN


In [171]:
def time_to_datetime(time):
    h = time.hour
    m = time.minute
    s = time.second
    return datetime.datetime(2024,8,3,h,m,s,tzinfo=datetime.timezone.utc)

In [172]:
import numpy as np

meas_time = raw.info["meas_date"]
df_action['开始时间'] = df_action['开始时间'].apply(lambda x: time_to_datetime(x))
onsets = (df_action['开始时间'] - meas_time).apply(lambda x: x.total_seconds())
durations = 20
descriptions = df_action['动作']
annot_action = mne.Annotations(
    onsets, durations, descriptions, orig_time=raw.info["meas_date"]
)

raw.set_annotations(raw.annotations + annot_action)

Measurement date,"August 03, 2024 18:02:15 GMT"
Experimenter,Unknown
Participant,X
Digitized points,Not available
Good channels,5 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,515.00 Hz
Highpass,0.00 Hz
Lowpass,257.50 Hz


In [173]:
raw.export(raw.filenames[0].replace('.edf', '_convert.edf'), overwrite=True)

## 数据不等长问题

In [174]:
515*(8*60+35)

265225

In [175]:
(515*(8*60+35))/500

530.45

In [176]:
(515*(8*60+35))/500 - 8*60+35

85.45000000000005